In [1]:
import time
import os
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    try:
      from google.colab import drive
      drive.mount('/content/drive/')
      %cd drive/My\ Drive/Project/project
      !git pull
    except OSError:
      !kill -9 -1

try:
    import psutil
    import humanize
    import os
    import GPUtil as GPU
except ModuleNotFoundError as e:
    print(e)
    !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
    !pip install gputil
    !pip install psutil
    !pip install humanize    
    
try:
  from read_data import system_load
  from dataset import load_dataset
  from classes import class_list
except ModuleNotFoundError as e:
  print(e)
  !pip install geopandas
finally:
  from read_data import system_load, save_obj, load_obj
  from dataset import load_dataset
  from classes import class_list
  clear_output()

import torch
import torchvision

meta, root_dir, _ = system_load(init=False)
datasets, dataset_lengths = load_dataset(meta, root_dir, include=[73, 18, 23, 19], workers=0)
save_obj(datasets["test"], "testing_object")
classes = class_list()

meta loaded!
correspondence of class names and labels: {73: 1, 18: 2, 23: 3, 19: 4}


In [ ]:
from train import train_model, model_creation, load_model
from evaluate import evaluate
device = "cuda"
model, optimizer, lr_scheduler, num_epochs = model_creation(num_epoch=50, device=device)
model = train_model(model, datasets, dataset_lengths, optimizer, lr_scheduler, num_epochs, device=device)
torch.save(model ,"model_final_v1")
model_loaded = load_model('model_final_v1')
#evaluate(model_loaded.to(device), datasets["test"], device, load_model=False, load_dataset=False)